In [2]:
import mysql.connector

mydb = mysql.connector.connect(
    host="127.0.0.1",
    user = "etl_user",
    password = "****"
)

my_cur = mydb.cursor()


In [3]:
my_cur.execute(
    "CREATE DATABASE IF NOT EXISTS pysparkdb"
)

In [4]:
my_cur.close()

True

In [5]:
mydb = mysql.connector.connect(
    host="127.0.0.1",
    user = "etl_user",
    password = "****",
    database = "pysparkdb"
)

my_cur = mydb.cursor()

my_cur.execute(
    """
    CREATE TABLE IF NOT EXISTS student(
        studentID VARCHAR(50) NOT NULL,
        name VARCHAR(50) NOT NULL,
        gender VARCHAR(50) NOT NULL
    )
    """
)

In [6]:
my_cur.execute("SHOW TABLES")

for x in my_cur:
    print(x)

('student',)


In [9]:
# insert records
sql = "INSERT INTO student (studentID, name, gender) VALUES (%s, %s, %s)"
val = [ 
    ("si1", "Robin", "M"),
    ("si2", "Maria", "F"),
    ("si3", "Julie", "F"),
    ("si4", "Bob", "M"),
    ("si6", "William", "M")
]

my_cur.executemany(sql,val)

In [10]:
mydb.commit()

In [11]:
my_cur.rowcount

5

In [12]:
my_cur.close()

True

In [15]:
import findspark
findspark.init()
import pyspark
import pandas as pd

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder.appName("JOIN-App").getOrCreate()

spark

### Read data from MySQL

In [16]:
cursor = mydb.cursor()
query = "SELECT * FROM student"
pdf = pd.read_sql(query, con=mydb)
cursor.close()

df = spark.createDataFrame(pdf)
df.show()

/tmp/ipykernel_26064/1826124330.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pdf = pd.read_sql(query, con=mydb)
/run/media/ashrulochan/Sector8/Books-and-Notes/To_be_continued/PySparkRecipes/spark_ttl_env/lib64/python3.10/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/run/media/ashrulochan/Sector8/Books-and-Notes/To_be_continued/PySparkRecipes/spark_ttl_env/lib64/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


+---------+-------+------+
|studentID|   name|gender|
+---------+-------+------+
|      si1|  Robin|     M|
|      si2|  Maria|     F|
|      si3|  Julie|     F|
|      si4|    Bob|     M|
|      si6|William|     M|
+---------+-------+------+



In [17]:
# if any unnecessary space or symbols are there use trim to clean data
student_df = df.select(trim(df.studentID), trim(df.name), trim(df.gender))
student_df.show()

+---------------+----------+------------+
|trim(studentID)|trim(name)|trim(gender)|
+---------------+----------+------------+
|            si1|     Robin|           M|
|            si2|     Maria|           F|
|            si3|     Julie|           F|
|            si4|       Bob|           M|
|            si6|   William|           M|
+---------------+----------+------------+



In [18]:
# rename column names
student_df = student_df.withColumnRenamed('trim(studentID)', 'studentID')\
    .withColumnRenamed('trim(name)', 'Name').withColumnRenamed('trim(gender)', 'Gender')

student_df.show()

+---------+-------+------+
|studentID|   Name|Gender|
+---------+-------+------+
|      si1|  Robin|     M|
|      si2|  Maria|     F|
|      si3|  Julie|     F|
|      si4|    Bob|     M|
|      si6|William|     M|
+---------+-------+------+



In [19]:
student_df.printSchema()

root
 |-- studentID: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Gender: string (nullable = true)



### Read data from JSON

In [25]:
json_data = {
    "studentID": ["si1", "si3", "si1", "si2", "si3", "si4", "si5", "si4", "si2"],
    "subject": ["Python", "Java", "Java", "Python", "Ruby", "C++", "C", "Python", "Java"]
}

sub_df = pd.DataFrame(json_data)
sub_df.to_json("subject.json")

In [20]:
from pyspark.sql import SQLContext

sql_context = SQLContext(spark.sparkContext)

/run/media/ashrulochan/Sector8/Books-and-Notes/To_be_continued/PySparkRecipes/spark_ttl_env/lib64/python3.10/site-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [28]:
# subject_df = sql_context.read.format("json").load("subject.json")
# subject_df.show()

df = pd.read_json("subject.json")
subject_df = spark.createDataFrame(df)
subject_df.show()

/run/media/ashrulochan/Sector8/Books-and-Notes/To_be_continued/PySparkRecipes/spark_ttl_env/lib64/python3.10/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/run/media/ashrulochan/Sector8/Books-and-Notes/To_be_continued/PySparkRecipes/spark_ttl_env/lib64/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


+---------+-------+
|studentID|subject|
+---------+-------+
|      si1| Python|
|      si3|   Java|
|      si1|   Java|
|      si2| Python|
|      si3|   Ruby|
|      si4|    C++|
|      si5|      C|
|      si4| Python|
|      si2|   Java|
+---------+-------+



In [29]:
subject_df.printSchema()

root
 |-- studentID: string (nullable = true)
 |-- subject: string (nullable = true)



In [30]:
# inner join
joined_df = subject_df.join(student_df, subject_df.studentID==student_df.studentID, how='inner')
joined_df.show()

+---------+-------+---------+-----+------+
|studentID|subject|studentID| Name|Gender|
+---------+-------+---------+-----+------+
|      si1| Python|      si1|Robin|     M|
|      si1|   Java|      si1|Robin|     M|
|      si2| Python|      si2|Maria|     F|
|      si2|   Java|      si2|Maria|     F|
|      si3|   Java|      si3|Julie|     F|
|      si3|   Ruby|      si3|Julie|     F|
|      si4|    C++|      si4|  Bob|     M|
|      si4| Python|      si4|  Bob|     M|
+---------+-------+---------+-----+------+



In [31]:
# or
joined_df = subject_df.join(student_df, on='studentID', how='inner')
joined_df.show()

+---------+-------+-----+------+
|studentID|subject| Name|Gender|
+---------+-------+-----+------+
|      si1| Python|Robin|     M|
|      si1|   Java|Robin|     M|
|      si2| Python|Maria|     F|
|      si2|   Java|Maria|     F|
|      si3|   Java|Julie|     F|
|      si3|   Ruby|Julie|     F|
|      si4|    C++|  Bob|     M|
|      si4| Python|  Bob|     M|
+---------+-------+-----+------+



In [32]:
# save inner joined data to json

joined_df.write.format('json').save('inner_joined_tbl')

In [33]:
# left outer join
left_outer_joined_df = subject_df.join(student_df, on = 'studentID', how='left_outer')
left_outer_joined_df.show()

+---------+-------+-----+------+
|studentID|subject| Name|Gender|
+---------+-------+-----+------+
|      si3|   Java|Julie|     F|
|      si1| Python|Robin|     M|
|      si2| Python|Maria|     F|
|      si1|   Java|Robin|     M|
|      si4|    C++|  Bob|     M|
|      si3|   Ruby|Julie|     F|
|      si5|      C| null|  null|
|      si2|   Java|Maria|     F|
|      si4| Python|  Bob|     M|
+---------+-------+-----+------+



In [34]:
# right outer join
right_outer_join_df = subject_df.join(student_df, on='studentID', how='right_outer')
right_outer_join_df.show()

+---------+-------+-------+------+
|studentID|subject|   Name|Gender|
+---------+-------+-------+------+
|      si1|   Java|  Robin|     M|
|      si1| Python|  Robin|     M|
|      si2|   Java|  Maria|     F|
|      si2| Python|  Maria|     F|
|      si3|   Ruby|  Julie|     F|
|      si3|   Java|  Julie|     F|
|      si4| Python|    Bob|     M|
|      si4|    C++|    Bob|     M|
|      si6|   null|William|     M|
+---------+-------+-------+------+



In [35]:
# full outer join
full_outer_joined = subject_df.join(student_df, on='studentID', how='outer')
full_outer_joined.show()

+---------+-------+-------+------+
|studentID|subject|   Name|Gender|
+---------+-------+-------+------+
|      si1| Python|  Robin|     M|
|      si1|   Java|  Robin|     M|
|      si2| Python|  Maria|     F|
|      si2|   Java|  Maria|     F|
|      si3|   Java|  Julie|     F|
|      si3|   Ruby|  Julie|     F|
|      si4|    C++|    Bob|     M|
|      si4| Python|    Bob|     M|
|      si5|      C|   null|  null|
|      si6|   null|William|     M|
+---------+-------+-------+------+

